# EDA To Prediction (DieTanic)
* Titanic dataset은 데이터 사이언스를 시작하는 초심자들이나 캐글 대회에 참여하려는 초보자들에게 매우 좋은 데이터셋이다. 
* 이 notebook의 목적은 예측 모델링 문제의 workflow가 어떤 것인지 아이디어를 제공하는 것이다. feature들을 확인하는 방법, 새로운 feature들과 머신러닝 컨셉을 추가하는 방법. 이 notebook은 초보자들이 모든 단계를 이해할 수 있도록 최대한 기본적으로 기록하였다.
---
## Contents of the Notebook:
### Part1 : Exploratory Data Analysis(EDA)
1. feature 분석
2. 여러 feature들을 고려해서 어떤 관계성이나 흐름을 파악한다.

### Part2 : Feature Engineering and Data Cleaning
1. 새로운 feature 추가
2. 불필요한 feature들 제거
3. 모델링에 적합한 feature로 변환하기

### Part3 : Predictive Modeling
1. Basic Modeling 돌리기
2. Cross Validation
3. Ensembling
4. Important Features Extraction

## Part1 : Exploratory Data Analysis(EDA)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
data = pd.read_csv('../input/titanic/train.csv')

In [3]:
data.head()

In [4]:
data.isnull().sum() # checking for total null values

Age, Cabin and Embarked은 결측값이 존재한다. 이것들을 수정해보자.

### How many Survived??

In [5]:
f, ax = plt.subplots(1,2, figsize=(18,8))
data['Survived'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Survived')
ax[0].set_ylabel('')
sns.countplot('Survived', data=data, ax=ax[1])
ax[1].set_title('Survived')
plt.show()

* 이 사고에서 살아남은 승객이 얼마 없다는 것이 명백해보인다.
* training set에 총 891명 중 오직 350명만 살아남았다. 훈련 데이터셋만 봤을때 이 충돌사고에서 오직 38.4%만 살아남은 것이다. 우리는 이 데이터로부터 더 좋은 인사이트를 얻기 위해서 그리고 어떤 범주의 승객들이 생존하고 사망했는지 알기위해서 더 파헤쳐봐야한다.
* 데이터셋의 다양한 feature들을 가지고 생존율을 확인해볼것이다.그 feature는 Sex, Port of Embarcation, Age 등 일것이다.

#### 먼저 다양한 타입의 feature들을 이해해보자!
---
## Types of Features
#### Categorical Features(범주형 특성):
범주형 변수는 두개 이상의 범주를 가진 데이터이고, 각각의 값들이 그 범주로 분류된다. 예를 들어, 성별은 두 개(남성과 여성)의 범주를 가진 범주형 변수이다. 이 상태로는 이러한 범주형 변수들을 정렬하거나 순서를 매기기 어렵다. 이런 변수들은 명목변수라고도 한다.
#### 범주형 features : Sex, Embarked

#### Ordinal Features(순서형 특성):
순서형 특성은 범주형 데이터들과 비슷한데, 차이점은 상대적인 순서나 값들을 정렬해줄 수 있다. 예를 들어, 크다, 중간, 작다라는 값들을 가지는 키와 같은 특성이 있을때, 키는 순서형 변수다. 여기서 우리는 변수의 상대적인 순서를 가진다.
#### 순서형 features : PClass

#### Continuous Feature(연속형 특성):
이 특성은 특성 컬럼에서 어떤 두 지점 사이의 값이거나 최소와 최대 사이의 값을 가질때 연속적이라고 할 수 있다. 
#### 연속형 features : Age
---
## Analysing the Features(특성 분석)
### Sex -> Categorical Feature

In [6]:
data.groupby(['Sex', 'Survived'])['Survived'].count()

In [7]:
f, ax = plt.subplots(1,2, figsize=(18,8))
data[['Sex', 'Survived']].groupby(['Sex']).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survivied vs Sex')
sns.countplot('Sex', hue='Survived', data=data, ax=ax[1])
ax[1].set_title('Sex : Survived vs Dead')
plt.show()

* 흥미로운 결과가 나왔다. 남자 승객수가 여자 승객수 보다 훨씬 많다. 그런데 생존자수는 여자가 남자의 거의 2배다. 남자 승객의 생존율이 약 18-19%인 반면에 여자 승객의 생존율은 대략 75%이다. 
* 여기서 Sex는 모델링하는데 매우 중요한 feature라는 걸 알 수 있다.
---
### Pclass -> Ordinal Feature

In [8]:
pd.crosstab(data.Pclass, data.Survived, margins=True).style.background_gradient(cmap='summer_r')

In [9]:
f, ax = plt.subplots(1,2, figsize=(18,8))
data['Pclass'].value_counts().plot.bar(color=['#CD7F32', '#FFDF00', '#D3D3D3'], ax=ax[0])
ax[0].set_title('Number of Passengers By Pclass')
ax[0].set_ylabel('Count')
sns.countplot('Pclass', hue='Survived', data=data, ax=ax[1])
ax[1].set_title('Pclass : Survived vs Dead')
plt.show()

* 사람들은 흔히 '돈으로 모든걸 살 수 없다'고 말한다. 그런데 우리는 여기서 Pclass 1의 승객들이 매우 높은 우선순위로 구조된것을 명백하게 알 수 있다. 물론 Pclass 3의 승객수가 훨씬 많긴하지만, 그래도 Pclass 3의 생존율은 고작 25%로 현저히 낮다.
* Pclass 1의 생존율은 약 63%이고 Pclass 2의 생존율은 약 48%인 것과 비교해서. 그래서 돈과 지위는 중요하다. 물질만능주의 세계란...
* 이제 좀 더 흥미로운 관찰을 해보자. Sex와 Pclass를 동시에 고려하여 생존율을 확인해보자!

In [10]:
pd.crosstab([data.Sex, data.Survived], data.Pclass, margins=True).style.background_gradient(cmap='summer_r')

In [11]:
sns.factorplot('Pclass', 'Survived', hue='Sex', data=data)
plt.show()

* 우리는 여기서 FactorPlot을 사용하였는데, 그 이유는 이 플랏이 범주형 데이터를 잘 구분해주기 때문이다.
* CrossTab이랑 FactorPlot을 보면, Pclass1의 여자 생존율은 거의 95-96% 이고, Pclass1에서 여자 승객 총 94명 중 오직 3명만 사망한 것을 쉽게 알 수 있다.
* 또한 Pclass와 상관없이, 구조 당시 여성이 우선순위였다는 것이 명백하다.
* 이것을 통해 Pclass 또한 중요한 특성인 것을 알 수 있다. 다른 특성들도 분석해보자!
---
### Age -> Continuous Feature

In [12]:
print('Oldest Passenger was of : ', data['Age'].max(), 'Years')
print('Youngest Passenger was of : ', data['Age'].min(), 'Years')
print('Average Age on the ship : ', data['Age'].mean(), 'Years')

In [13]:
f, ax = plt.subplots(1,2, figsize=(18,8))
sns.violinplot('Pclass', 'Age', hue='Survived', data=data, split=True
               , ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0, 110, 10))
sns.violinplot('Sex', 'Age', hue='Survived', data=data, split=True, ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0, 110, 10))
plt.show()

##### Observaions:
1) 아이들의 수는 Pclass가 높을수록(3으로 갈수록) 증가하고, 10살이하 승객들의 생존율은 Pclass와 상관없어 보인다.  
2) Pclass1에서 20-50대의 생존율은 매우 높고 심지어 여성의 생존율보다 더 높다.  
3) 남섬의 경우, 나이가 많을수록 생존율이 줄어든다.  

* 위에서도 봤듯이, Age feature는 177개의 결측값을 가지고 있다. 이 결측치들을 채우기 위해서 age 데이터셋의 평균값을 사용할 수 있다.
* 그러나 문제는 나이층이 매우 광범위하고 넓게 퍼져있다. 4살 아이의 나이를 평균나이인 29살로 대체할 수는 없다. 어떤 연령대가 속하는지 알 수 있는 방법이 있을까?
* 그렇다! 방법은 바로 Name feature를 확인해보는 것이다. 이름 데이터들을 보면, Mr나 Mrs와 같은 인삿말을 볼 수 있다. 따라서 Mr과 Mrs의 평균나이를 관련있는 그룹에 매칭해주면 된다.
---
### "What is in a Name" -> Feature :p

In [14]:
data['Initial'] = 0
for i in data:
    data['Initial'] = data.Name.str.extract('([A-Za-z]+)\.')
# 인삿말을 추출해보자!

* 여기서 우리는 Regex(정규표현식):[A-za-z]+)...를 사용할 것이다. 이게 뭐냐면, A-z or a-z와 마지막에 .(dot)으로 표기된 문자열을 찾는 것이다. 이것을 통해 우리는 Name으로부터 Initial을 성공적으로 추출할 수 있다.

In [15]:
pd.crosstab(data.Initial, data.Sex).T.style.background_gradient(cmap='summer_r')
# Initial과 Sex를 확인해보자!

* 여기서 Mlle이나 Mme과 같이 Miss를 나타내는데 잘 못 표기된 Initial들이 있다. 이 데이터들은 Miss로 바꿔주고 다른 값들도 이런식으로 바꿔준다.

In [16]:
data['Initial'].replace(['Mlle', 'Mme', 'Ms', 'Dr', 'Major', 'Lady', 'Countess', 'Jonkheer', 'Col', 'Rev', 'Capt', 'Sir', 'Don'],
                       ['Miss', 'Miss', 'Miss', 'Mr', 'Mr', 'Mrs', 'Mrs', 'Other', 'Other', 'Other', 'Mr', 'Mr', 'Mr'], inplace=True)

In [17]:
data.groupby('Initial')['Age'].mean()
# Initial에 따른 평균 나이를 확인해보자!

### Age의 결측치를 채워보자

In [18]:
# 결측치들을 올림한 평균나이로 채워주자
data.loc[(data.Age.isnull())&(data.Initial=='Mr'), 'Age'] = 33
data.loc[(data.Age.isnull())&(data.Initial=='Mrs'), 'Age'] = 36
data.loc[(data.Age.isnull())&(data.Initial=='Master'), 'Age'] = 5
data.loc[(data.Age.isnull())&(data.Initial=='Miss'), 'Age'] = 22
data.loc[(data.Age.isnull())&(data.Initial=='Other'), 'Age'] = 46

In [19]:
data.Age.isnull().any()
# 이제 Age에 null값이 없을 것이다!

In [20]:
f, ax = plt.subplots(1,2, figsize=(20,10))
data[data['Survived']==0].Age.plot.hist(ax=ax[0], bins=20, edgecolor='black', color='red')
ax[0].set_title('Survived = 0')
x1 = list(range(0, 85, 5))
ax[0].set_xticks(x1)
data[data['Survived']==1].Age.plot.hist(ax=ax[1], color='green', bins=20, edgecolor='black')
ax[1].set_title('Survived = 1')
x2 = list(range(0, 85, 5))
ax[1].set_xticks(x2)
plt.show()

##### Observations:
1) 영유아(5세미만)들은 상당수 구조되었다.(여성과 아동 우선 정책)  
2) 최고령자(80세)도 구조되었다.  
3) 사망자수가 가장 작은 그룹은 30-40대이다.  

In [21]:
sns.factorplot('Pclass', 'Survived', col='Initial', data=data)
plt.show()

* 여성과 아동 우선 정책은 Pclass와 상관없이 유효하다.
---
### Embarked -> Categorical Value

In [22]:
pd.crosstab([data.Embarked, data.Pclass], [data.Sex, data.Survived], margins=True).style.background_gradient(cmap='summer_r')

#### Chances for Survival by Port of Embarkation

In [23]:
sns.factorplot('Embarked', 'Survived', data=data)
fig = plt.gcf()
fig.set_size_inches(5,3)
plt.show()

* Port C에서의 생존율이 약 0.55로 가장 높고 S에서 가장 낮다.

In [24]:
f, ax = plt.subplots(2,2, figsize=(20,15))
sns.countplot('Embarked', data=data, ax=ax[0,0])
ax[0,0].set_title('No. of Passengers Boarded')
sns.countplot('Embarked', hue='Sex', data=data, ax=ax[0,1])
ax[0,1].set_title('Male-Female Split for Embarked')
sns.countplot('Embarked', hue='Survived', data=data, ax=ax[1,0])
ax[1,0].set_title('Embarked vs Survived')
sns.countplot('Embarked', hue='Pclass', data=data, ax=ax[1,1])
ax[1,1].set_title('Embarked vs Pclass')
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

#### Observations:
1) S에서 탑승한 최다 승객수의 대다수가 Pclass 3에 속한다.   
2) C의 승객 중 상당수가 살아남았기 때문에 운이 좋아 보인다. 그 이유는 구조된 이들이 모두 Pclass1과 Pclass2이기 때문일 것이다.  
3) S 항구에서는 대다수 부자들이 탑승한 것 같다. 그럼에도 여전히 이 항구에서 탑승한 승객들의 생존율은 낮다, 그 이유는 Pclass3의 많은 승객들 대략 81%가 구조되지 못했기 때문이다.  
4) Q 항구의 승객들은 약 95%가 Pclass3이다. 

In [25]:
sns.factorplot('Pclass', 'Survived', hue='Sex', col='Embarked', data=data)
plt.show()

#### Observations:
1) Pclass와 관계없이 Pclass1과 Pclass2의 여성 생존율은 거의 1이다.  
2) S 항구에서 탑승한 Pclass3의 남성과 여성 승객 모두 생존율이 매우 낮은 걸로 보아 매우 불행한 것 같다.(돈이 중요하다....)   
3) Q 항구에서 탑승한 남성의 경우 거의 모든이들이 Pclass3이기에 매우 불행한 결론을 보인다.  
---
### Filling Embarked NaN
* 승객의 대다수가 S 항구에서 탑승했으므로, 우리는 Embarked NaN값을 S로 채워주자. 

In [26]:
data['Embarked'].fillna('S', inplace=True)

In [27]:
data.Embarked.isnull().any()
# 드디어 NaN 값을 없앴다!

### SibSp -> Discrete Feature
* 이 특성은 해당 승객이 혼자인지 그의 가족구성원과 함께인지를 알려준다.
* Sibling = brother, sister, stepbrother, stepsister
* Spouse = husband, wife

In [28]:
pd.crosstab([data.SibSp], data.Survived).style.background_gradient(cmap='summer_r')

In [29]:
f, ax = plt.subplots(1,2, figsize=(20,8))
sns.barplot('SibSp', 'Survived', data=data, ax=ax[0])
ax[0].set_title('SibSp vs Survived')
sns.pointplot('SibSp', 'Survived', data=data, ax=ax[1])
ax[1].set_title('SipSb vs Survived')
plt.close(2)
plt.show()

In [30]:
pd.crosstab(data.SibSp, data.Pclass).style.background_gradient(cmap='summer_r')

#### Observations:
* barplot과 pointplot은 해당승객이 형제자매 없이 혼자 탑승한 경우라면 그의 생존율은 34.5%이라는 것을 알려준다. 형제자매 수가 증가하면 그래프는 대체적으로 감소한다. 이건 당연하다. 왜냐하면 만약 내 가족이 같이 탑승해 있다면, 내 목숨보다 가족의 목숨을 우선시했을테니까. 놀랍게도 5-8명의 가족구성원의 경우 생존율이 0%이다. 아마 이유는 Pclass와 관련있지 않을까?
* 그 이유는 Pclass가 맞다. 위의 crosstab을 보면, SibSp>3인 승객의 경우 모두 Pclass가 3이다. 이것은 Pclass3에 속한 모든 대가족(4인이상)이 사망하는 것에 임박하다는 뜻이다.
---
### Parch

In [31]:
pd.crosstab(data.Parch, data.Pclass).style.background_gradient(cmap='summer_r')

* crosstab은 여기서 한번 더 대가족의 경우 Pclass3인걸 알려준다.

In [32]:
f, ax = plt.subplots(1,2, figsize=(20,8))
sns.barplot('Parch', 'Survived', data=data, ax=ax[0])
ax[0].set_title('Parch vs Survived')
sns.pointplot('Parch', 'Survived', data=data, ax=ax[1])
ax[1].set_title('Parch vs Survived')
plt.close(2)
plt.show()

#### Observations:
* 여기서도 결과가 비슷하다. 부모와 동승한 승객의 경우 생존율이 높다. 그렇지만 구성원 수가 증가할수록 생존율은 감소한다.
* 부모가 1-3명 정도인 경우는 생존율이 높다. 그러나 혼자인 경우도 치명적이고 부모가 4명 초과인 경우도 생존율이 감소한다.
---
### Fare -> Continuous Feature

In [33]:
print('Highest Fare was:', data['Fare'].max())
print('Lowest Fare was:', data['Fare'].min())
print('Average Fare was:', data['Fare'].mean())

In [34]:
f, ax = plt.subplots(1,3, figsize=(20,8))
sns.distplot(data[data['Pclass']==1].Fare, ax=ax[0])
ax[0].set_title('Fares in Pclass 1')
sns.distplot(data[data['Pclass']==2].Fare, ax=ax[1])
ax[1].set_title('Fares in Pclass 2')
sns.distplot(data[data['Pclass']==3].Fare, ax=ax[2])
ax[2].set_title('Fares in Pclass 3')
plt.show()

* Pclass1에서 승객들의 요금 차이가 크다. 이 분포는 표준이 감소함에 따라 계속 감소한다. 이것도 연속형 자료이므로 구간화를 사용하여 이산값으로 변환할 수 있다. 
---
#### Observations in a Nutshell for all features:
* Sex : 남성과 비교해서 여성의 생존율이 높다.
* Pclass : 1등석 승객이 되면 생존 가능성이 더 높아진다는 가시적인 추세가 있다. 3등석의 생존율은 매우 낮다. 여성의 경우 1등석의 생존확률은 거의 1이고 2등석의 생존율도 높다. 돈은 승리한다!!!
* Age : 5-10세의 어린이는 생존 가능성이 높다. 15-35세 사이의 승객이 많이 사망했다.
* Embarked : 매우 흥미로운 특성이다. 1등석 승객의 대다수가 S에서 탑승했음에도 불구하고 C에서 생존할 가능성이 더 좋아 보인다. Q의 승객은 모두 3등석이다.
* Parch+SibSp : 형제자매가 1-2명, 배우자가 있거나 부모가 1-3명인 경우 혼자이거나 대가족이 함께 여행하는 것보다 생존 가능성이 더 높다.
___
### Correlation Between The Features

In [35]:
sns.heatmap(data.corr(), annot=True, cmap='RdYlGn', linewidth=0.2)
# data.corr() -> correaltion matrix
fig=plt.gcf()
fig.set_size_inches(10,8)
plt.show()

#### Heatmap 해석
* 가장 먼저 주목해야 할 것은 알파벳이나 문자열 사이의 상관관계를 알 수 엉ㅂㅅ다는 것이 분명하기 때문에 숫자 데이터만 비교된다는 것이다. 플롯을 이해하기 전에 상관관계가 정확히 무엇인지 살펴보겠습니다.   
* Positive Correlation : 특성A의 증가가 특성 B의 증가로 이어진다면 양의 상관관계가 있다. 값1은 완전한 양의 상관관계를 의미한다.
* Negative Correlation : 특성A가 증가할때 특성B가 감소하면 음의 상관관계가 있다. 값 -1은 완전히 음의 상관관계를 의미한다.
* 이제 두 특성이 크게 또는 완전히 상관관계가 있으므로 하나의 증가가 다른 특성의 증가로 이어진다고 가정해 보겠습니다. 이는 두 기능 모두 매우 유사한 정보를 포함하고 있으며 정보의 변동이 거의 혹은 전혀 없음을 의미한다. 둘 다 거의 동일한 정보를 포함하므로 이를 다중공선성(MultiColinearity)이라고 한다.
* 둘 중 하나가 중복되므로 둘 다 사용해야 한다고 생각하나요? 모델을 만들거나 훈련하는 동안 훈련 시간과 많은 이점을 줄이므로 중복 특성을 제거하려고 해야한다.
* 이제 위의 히트맵에서 특성이 그다지 상관관계가 없음을 알 수 있다. 가장 높은 상관관계는 SibSp와 Parch 사이, 즉 0.41이다. 따라서 모든 특성을 계속 사용할 수 있다. 
___
## Part2 : Feature Engineering and Data Cleaning
* Feature Engineering 이란?
* 특성이 있는 데이터 세트가 제공될 때마다 모든 특성이 중요할 필요는 없다. 제거해야할 중복 특성이 많이 있을 수 있다. 또한 다른 특성에서 정보를 관찰하거나 추출하여 새로운 기능을 얻거나 추가할 수 있다.
* Name 특성을 사용하여 Initial 특성을 가져오는 것은 한 에시다. 새로운 특성을 얻고 몇가지를 제거할 수 있는지 보자. 또한 기존 관련 특성을 Predicitve Modeling에 적합한 형식으로 변환할 것이다.
---
### Age_band
* Age Feature의 문제점:
* 나이는 연속적인 특성이라고 앞서 언급했듯이 기계 학습 모델의 연속 변수에는 문제가 있다. 예) 스포츠인을 성별로 그룹화하거나 정렬하면 쉽게 남성과 여성을 구분할 수 있다.
* 이제 연령별로 그룹화하라고 하면 어떻게 해야할까? 30명의 사람이 있는 경우 30개의 연령 값이 있을 수 있다. 이제 이것이 문제가 된다.
* 그룹화와 정규화를 통해 이러한 연속값을 범주형값으로 변환해야 한다. 여기서는 그룹화를 사용할 것이다. 즉, 연령대 범위를 단일 그룹으로 그룹화하거나 단일값을 할당할 것이다.
* 승객의 최대 연령은 80세였다. 따라서 0-80의 범위를 5개의 그룹으로 나눈다. 80/5=16이다. 크기가 16인 그룹.

In [36]:
data['Age_band']=0
data.loc[data['Age']<=16, 'Age_band']=0
data.loc[(data['Age']>16)&(data['Age']<=32), 'Age_band']=1
data.loc[(data['Age']>32)&(data['Age']<=48), 'Age_band']=2
data.loc[(data['Age']>48)&(data['Age']<=64), 'Age_band']=3
data.loc[data['Age']>64, 'Age_band']=4
data.head(2)

In [37]:
data['Age_band'].value_counts().to_frame().style.background_gradient(cmap='summer')
# 각 그룹의 수를 확인해보자

In [38]:
sns.factorplot('Age_band','Survived', data=data, col='Pclass')
plt.show()

* Pclass와 상관없이 나이가 증가할수록 생존율이 떨어진다.
---
### Family_Size and Alone
* 여기서 우리는 'Family_size'와 'Alone'이라는 새로운 특성을 만들고 분석할 것이다. 이 특성은 Parch와 SibSp의 합이다. 생존율이 승객의 가족 규모와 관련이 있는지 확인할 수 있도록 결합된 데이터를 제공한다. Alone은 승객이 혼자인지 여부를 나타낸다.

In [39]:
data['Family_Size']=0
data['Family_Size']=data['Parch']+data['SibSp']
data['Alone']=0
data.loc[data.Family_Size==0, 'Alone']=1

f, ax = plt.subplots(1,2, figsize=(18,6))
sns.pointplot('Family_Size','Survived', data=data, ax=ax[0])
ax[0].set_title('Family_Size vs Survived')
sns.pointplot('Alone', 'Survived', data=data, ax=ax[1])
ax[1].set_title('Alone vs Survived')
plt.close(2)
plt.close(3)
plt.show()

##### Family_Size=0 은 승객이 혼자인 걸 의미한다.
* 분명히 혼자이거나 family_size=0이라면 생존 가능성은 매우 낮다. 가족규모가 5인이상인 경우도 줄어든다. 이것 또한 모델의 중요한 특성으로 보인다. 이것을 더 조사해보자.

In [40]:
sns.factorplot('Alone', 'Survived', data=data, hue='Sex', col='Pclass')
plt.show()

* 혼자일 확률이 높은 Pclass3을 제외하고는 혼자 있는 여성이 가족이 있는 여성보다 성별이나 Pclass를 불문하고 혼자 있다는 것이 좋지 않다는 것을 알 수 있다.
---
### Fare_Range
* 요금도 연속적인 특성이므로 서수값으로 변환해야 한다. 이를 위해 우리는 pandas.qcut을 사용할 것이다.
* 따라서 qcut이 하는 일은 우리가 전달한 그룹의 수에 따라 값을 분할하거나 정렬하는 것이다. 따라서 5개의 그룹에 대해 전달하면 5개의 개별 그룹 또는 값 범위에 균등한 간격으로 값을 정렬한다.

In [41]:
data['Fare_Range']=pd.qcut(data['Fare'],4)
data.groupby(['Fare_Range'])['Survived'].mean().to_frame().style.background_gradient(cmap='summer_r')

* 위에서 논의한 바와 같이, 요금 범위가 증가함에 따라 생존 가능성이 증가한다는 것을 분명히 알 수 있다.
* 이제 Fare_Range값을 그대로 전달할 수 없다. Age_Band에서 했던 것과 같은 싱글톤 값으로 변환해야 한다. 

In [42]:
data['Fare_cat']=0
data.loc[data['Fare']<=7.91,'Fare_cat']=0
data.loc[(data['Fare']>7.91)&(data['Fare']<=14.454), 'Fare_cat']=1
data.loc[(data['Fare']>14.454)&(data['Fare']<=31), 'Fare_cat']=2
data.loc[(data['Fare']>31)&(data['Fare']<=513), 'Fare_cat']=3

In [43]:
sns.factorplot('Fare_cat', 'Survived', data=data, hue='Sex')
plt.show()

* 분명히 Fare_cat이 증가함에 따라 생존율이 증가한다. 이 특성은 Sex와 함께 모델링시 중요한 특성이 될 수 있다.
---
### Converting String Values into Numeric
* 기계 학습 모델에 문자열을 전달할 수 없기 때문에 Sex, Embarked 등과 같은 기능을 숫자 값으로 변환해야 한다.

In [44]:
data['Sex'].replace(['male', 'female'],[0,1],inplace=True)
data['Embarked'].replace(['S','C','Q'],[0,1,2], inplace=True)
data['Initial'].replace(['Mr', 'Mrs','Miss','Master','Other'],[0,1,2,3,4], inplace=True)

#### 불필요한 특성 삭제
* Name -> 범주형 값으로 변환할 수 없으므로 필요없다.
* Age -> Age_band 특성이 있으므로 필요없다.
* Ticket -> 분류할 수 없는 임의의 문자열이다.
* Fare -> Fare_cat 특성이 있으므로 필요없다.
* Cabin -> 많은 NaN값과 많은 승객이 여러 개의 객실을 가지고 있다. 따라서 쓸모없는 특성이다.
* Fare_Range -> Fare_cat 특성이 있다.
* Passengerid -> 분류할 수 없다.

In [45]:
data.drop(['Name','Age','Ticket','Fare','Cabin','Fare_Range','PassengerId'], axis=1, inplace=True)
sns.heatmap(data.corr(), annot=True, cmap='RdYlGn', linewidth=0.2, annot_kws={'size':20})
fig=plt.gcf()
fig.set_size_inches(18,15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

* 이제 위의 상관 플롯에서 양성으로 관련된 몇 가지 특성을 볼 수 있다. 그들 중 일부는 SibSp와 Family_Size, Parch와 Family_size이고 일부는 Alone과 Family_Size 같은 음성으로 관련된 것이다.
---
## Part3 : Predictive Modeling
* 우리는 EDA 부분에서 몇 가지 통찰력을 얻었다. 그러나 그것으로는 승객이 생존할지 사망할지 정확하게 예측하거나 말할 수 없다. 이제 우리는 훌륭한 분류 알고리즘을 사용하여 승객이 생존할지 여부를 예측할 것이다. 아래는 모델을 만드는데 사용할 알고리즘이다.
    * Logistic Regression
    * Support Vector Machines(Linear and Radial)
    * Random Forest
    * K-Nearest Neighbouts
    * Naive Bayes
    * Decision Tree

In [46]:
# importing all the required ML packages
from sklearn.linear_model import LogisticRegression # logistic regression
from sklearn import svm # support vector machine
from sklearn.ensemble import RandomForestClassifier # Random Forest
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.naive_bayes import GaussianNB # Naive Bayes
from sklearn.tree import DecisionTreeClassifier # Decision Tree
from sklearn.model_selection import train_test_split # training and testing data split
from sklearn import metrics # accuracy measure
from sklearn.metrics import confusion_matrix # for confusion matrix


In [47]:
train, test=train_test_split(data, test_size=0.3, random_state=0, stratify=data['Survived'])
train_X=train[train.columns[1:]]
train_Y=train[train.columns[:1]]
test_X=test[test.columns[1:]]
test_Y=test[test.columns[:1]]
X=data[data.columns[1:]]
Y=data['Survived']

### Radial Support Vector Machines(rbf-SVM)

In [48]:
model=svm.SVC(kernel='rbf', C=1, gamma=0.1)
model.fit(train_X, train_Y)
prediction1=model.predict(test_X)
print('Accuracy for rbf SVM is ', metrics.accuracy_score(prediction1, test_Y))

### Linear Support Vector Machine(linear-SVM)

In [49]:
model=svm.SVC(kernel='linear', C=0.1, gamma=0.1)
model.fit(train_X, train_Y)
prediction2=model.predict(test_X)
print('Accuracy for linear SVM is', metrics.accuracy_score(prediction2, test_Y))

### Logistic Regression

In [50]:
model=LogisticRegression()
model.fit(train_X, train_Y)
prediction3=model.predict(test_X)
print('The accuracy of the Logistic Regression is', metrics.accuracy_score(prediction3, test_Y))

### Decision Tree

In [51]:
model=DecisionTreeClassifier()
model.fit(train_X, train_Y)
prediction4=model.predict(test_X)
print('The accuracy of the Decision Tree is', metrics.accuracy_score(prediction4, test_Y))

### K-Nearest Neighbours(KNN)

In [52]:
model=KNeighborsClassifier()
model.fit(train_X, train_Y)
prediction5=model.predict(test_X)
print('The accuracy of the KNN is', metrics.accuracy_score(prediction5, test_Y))

* 이제 n_neighbours 속성의 값을 변경함에 따라 KNN 모델의 정확도가 변한다. 기본값은 5이다. 다양한 n_neighbours값에 대한 정확도를 확인하자.

In [53]:
a_index=list(range(1,11))
a=pd.Series()
x=[0,1,2,3,4,5,6,7,8,9,10]
for i in list(range(1,11)):
    model=KNeighborsClassifier(n_neighbors=i)
    model.fit(train_X, train_Y)
    prediction=model.predict(test_X)
    a=a.append(pd.Series(metrics.accuracy_score(prediction, test_Y)))
plt.plot(a_index, a)
plt.xticks(x)
fig=plt.gcf()
fig.set_size_inches(12,6)
plt.show()
print('Accuracies for different values of n are:', a.values, 'with the max value as ', a.values.max())

### Gaussian Naive Bayes

In [54]:
model=GaussianNB()
model.fit(train_X, train_Y)
prediction6=model.predict(test_X)
print('The accuracy of the NaiveBayes is', metrics.accuracy_score(prediction6, test_Y))

### Random Forests

In [55]:
model=RandomForestClassifier(n_estimators=100)
model.fit(train_X, train_Y)
prediction7=model.predict(test_X)
print('The accuracy of the Random Forests is', metrics.accuracy_score(prediction7, test_Y))

* 모델의 정확도가 분류기의 견고성을 결정하는 유일한 요소는 아니다. 분류기가 훈련 데이터에 대해 훈련되고 테스트 데이터에 대해 테스트되었으며 90%의 정확도를 기록했다고 가정해보자!
* 이제 분류기의 정확도가 매우 좋은 것 같지만, 모든 새로운 테스트 세트에 대해 90%의 정확도를 확인할 수 있을까요? 대답은 아니오이다. 분류기가 자체 학습에 사용할 모든 인스턴스를 결정할 수 없기 때문이다. 훈련 및 테스트 데이터가 변경되면 정확도도 변경된다. 증가하거나 감소할 수 있다. 이것을 모델 분산이라고 한다.
* 이를 극복하고 일반화된 모델을 얻기 위해 Cross Validation(교차검증)을 사용한다.
---
### Cross Validation
* 대다수의 경우 데이터가 불균형하다. 즉, class1 인스턴스의 수는 많지만 다른 class 인스턴스의 수는 적을 수 있다. 따라서 데이터 세트의 모든 인스턴스에 대해 알고리즘을 훈련하고 테스트해야 한다. 그런 다음 데이터 세트에 대해 언급된 모든 정확도의 평균을 취할 수 있다.   
* 1) K-Fold 교차 검증은 먼저 데이터 세트를 k-subsets으로 나누어 작동한다.      
* 2) 데이터 세트를 k=5 로  나눈다. 테스트를 위해 1개를 놔두고 4개는 알고리즘을 훈련한다.       
* 3) 각 반복마다 테스트 파트를 변경하고 다른 파트에 대해 알고리즘을 훈련하여 프로세스를 계속한다. 그런 다음 정확도와 오류를 평균하여 알고리즘의 평균 정확도를 얻는다.       
* 이것을 K-Fold 교차 검증이라고 한다.
* 4) 알고리즘은 일부 훈련 데이터에 대한 데이터 세트에 대해 과소적합될 수 있고 때로는 다른 훈련 세트에 대한 데이터에 과대적합될 수도 있다. 따라서 교차검증을 통해 일반화된 모델을 얻을 수 있다.

In [56]:
from sklearn.model_selection import KFold # for K-fold cross validation
from sklearn.model_selection import cross_val_score # score evaluation
from sklearn.model_selection import cross_val_predict # prediction
kfold = KFold(n_splits=10, random_state=22) # k=10, split the data into 10 equal parts
xyz = []
accuracy = []
std = []
classifiers=['Linear Svm', 'Radial Svm', 'Logistic Regression', 'KNN', 'Decision Tree','Naive Bayes', 'Random Forest']
models=[svm.SVC(kernel='linear'), svm.SVC(kernel='rbf'), 
        LogisticRegression(), KNeighborsClassifier(n_neighbors=9),
        DecisionTreeClassifier(), GaussianNB(), RandomForestClassifier(n_estimators=100)]
for i in models:
    model = i
    cv_result=cross_val_score(model, X, Y, cv=kfold, scoring='accuracy')
    cv_result=cv_result
    xyz.append(cv_result.mean())
    std.append(cv_result.std())
    accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz, 'Std':std}, index=classifiers)
new_models_dataframe2

In [57]:
plt.subplots(figsize=(12,6))
box=pd.DataFrame(accuracy, index=[classifiers])
box.T.boxplot()

In [58]:
new_models_dataframe2['CV Mean'].plot.barh(width=0.8)
plt.title('Average CV Mean Accuracy')
fig=plt.gcf()
fig.set_size_inches(8,5)
plt.show()

* 분류 정확도는 불균형으로 인해 때때로 오해의 소지가 있다. 모델이 어디에서 잘못되었는지 또는 모델이 어떤 클래스를 잘못 예측했는지 보여주는 혼동 행렬의 도움으로 요약된 결과를 얻을 수 있다. 
---
### Confusion Matrix(혼동행렬)
* 분류기가 만든 옳고 그른 분류의 수를 제공한다.

In [59]:
f,ax=plt.subplots(3,3,figsize=(12,10))
y_pred = cross_val_predict(svm.SVC(kernel='rbf'),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,0],annot=True,fmt='2.0f')
ax[0,0].set_title('Matrix for rbf-SVM')
y_pred = cross_val_predict(svm.SVC(kernel='linear'),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,1],annot=True,fmt='2.0f')
ax[0,1].set_title('Matrix for Linear-SVM')
y_pred = cross_val_predict(KNeighborsClassifier(n_neighbors=9),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,2],annot=True,fmt='2.0f')
ax[0,2].set_title('Matrix for KNN')
y_pred = cross_val_predict(RandomForestClassifier(n_estimators=100),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,0],annot=True,fmt='2.0f')
ax[1,0].set_title('Matrix for Random-Forests')
y_pred = cross_val_predict(LogisticRegression(),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,1],annot=True,fmt='2.0f')
ax[1,1].set_title('Matrix for Logistic Regression')
y_pred = cross_val_predict(DecisionTreeClassifier(),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,2],annot=True,fmt='2.0f')
ax[1,2].set_title('Matrix for Decision Tree')
y_pred = cross_val_predict(GaussianNB(),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[2,0],annot=True,fmt='2.0f')
ax[2,0].set_title('Matrix for Naive Bayes')
plt.subplots_adjust(hspace=0.2, wspace=0.2)
plt.show()

#### Interpreting Confusion Matrix
* 왼쪽 대각선은 각 클래스에 대해 수행된 올바른 예측의 수를 나타내고 오른쪽 대각선은 잘못된 예측의 수를 나타낸다. rbf-SVM의 첫 번째 플롯을 고려해보자.
* 정확한 예측수는 491(사망) + 247(생존)이며 평균 CV정확도는 (491+247)/891 = 82.8% 이다.
* 오류 -> 58명의 사망자를 생존자로, 95명이 사망한 것으로 잘 못 분류했다. 따라서 죽은 사람을 생존자로 예측함으로써 더 많은 실수를 저질렀다.
* 모든 행렬을 살펴보면 rbf-SVM이 사망한 승객을 정확하게 예측할 확률이 더 높지만 NaiveBayes가 생존한 승객을 정확하게 예측할 확룰이 더 높다고 말할 수 있다.
---
#### Hyper-Parameters Tuning
* 기계 학습 모델은 블랙박스와 같다. 이 블랙박스에 대한 몇가지 기본 매개변수 값이 있으며 더 나은 모델을 얻기 위해 조정하거나 변경할 수 있다. SVM 모델의 C 및 gamma와 마찬가지로 분류기에 대한 유사하게 다른 매개변수를 하이퍼 파라미터라고 하며 알고리즘의 학습률을 변경하고 더 나은 모델을 얻기 위해 조정할 수 있다. 이것을 하이퍼파라미터 튜닝이라고 한다.
* 우리는 2개의 최고의 분류기, 즉 SVM과 RandomForests에 대한 하이퍼파라미터를 조정할 것이다.
---
### SVM

In [60]:
from sklearn.model_selection import GridSearchCV
C=[0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
gamma=[-.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
kernel=['rbf','linear']
hyper={'kernel':kernel, 'C':C, 'gamma':gamma}
gd=GridSearchCV(estimator=svm.SVC(), param_grid=hyper, verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

### Random Forests

In [61]:
n_estimators=range(100,1000,100)
hyper={'n_estimators':n_estimators}
gd=GridSearchCV(estimator=RandomForestClassifier(random_state=0), param_grid=hyper, verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

* Rbf-SVM에 대한 최고 점수는 C=0.4 및 gamma=0.3 일 때 82.82%이다.
* RandomForest의 경우 점수는 n_estimators=300일 때 약 81.9%이다.
---
## Ensembling
* 앙상블은 모델의 정확도나 성능을 높이는 좋은 방법입니다. 간단히 말해서 여러 가지 단순한 모델을 조합하여 하나의 강력한 모델을 만드는 것입니다.
* 우리가 전화기를 구매하고 싶고 다양한 매개변수를 기반으로 많은 사람들에게 전화기 정보에 대해 물어보고 싶다고 가정해 보자. 따라서 다양한 매개변수를 모두 분석한 후 단일 제품에 대해 강력한 판단을 내릴 수 있다. 모델의 안정성을 향상시키는 이것이 앙상블이다. 앙상블은 다음과 같은 방법으로 수행할 수 있다. :   
1) Voting Classifier  
2) Bagging   
3) Boosting  
---
### Voting Classifier
* 다양한 단순 기계학습모델의 예측을 결합하는 가장 간단한 방법이다. 모든 하위 모델의 예측을 기반으로 평균 예측결과를 제공한다. 하위 모델 또는 기본 모델은 모두 다른 유형이다.

In [62]:
from sklearn.ensemble import VotingClassifier
ensemble_lin_rbf =VotingClassifier(estimators=[('KNN', KNeighborsClassifier(n_neighbors=10)),
                                              ('RBF', svm.SVC(probability=True,kernel='rbf', C=0.5,gamma=0.1)),
                                              ('RFor', RandomForestClassifier(n_estimators=500, random_state=0)),
                                              ('LR', LogisticRegression(C=0.05)),
                                              ('DT', DecisionTreeClassifier(random_state=0)),
                                              ('NB', GaussianNB()),
                                              ('svm', svm.SVC(kernel='linear', probability=True))], voting='soft').fit(train_X, train_Y)
print('The accuracy for ensembled model is:', ensemble_lin_rbf.score(test_X, test_Y))
cross=cross_val_score(ensemble_lin_rbf, X, Y, cv=10, scoring='accuracy')
print('The cross validated score is', cross.mean())

### Bagging
* Bagging은 일반적인 앙상블 방법이다. 데이터 세트의 작은 파티션에 유사한 분류기를 적용한 다음 모든 예측의 평균을 취하여 작동한다. 평균화로 인해 분산이 감소한다. Voting Classifier와 달리 Bagging은 유사한 classifier를 사용한다.
---
#### Bagged KNN
* Bagging은 분산이 높은 모델에서 가장 잘 작동한다. 이에 대한 예로는 Decision Tree or Random Forest가 있다. n_neighbors의 작은 값으로 KNN을 n_neighbors의 작은 값으로 사용할 수 있다.

In [63]:
from sklearn.ensemble import BaggingClassifier
model=BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=3), random_state=0, n_estimators=700)
model.fit(train_X, train_Y)
prediction=model.predict(test_X)
print('The accuracy for bagged KNN is:', metrics.accuracy_score(prediction, test_Y))
result=cross_val_score(model, X, Y, cv=10, scoring='accuracy')
print('The cross validated score for bagged KNN is:', result.mean())

#### Bagged DecisionTree

In [64]:
model=BaggingClassifier(base_estimator=DecisionTreeClassifier(), random_state=0, n_estimators=100)
model.fit(train_X, train_Y)
prediction=model.predict(test_X)
print('The accuracy for bagged Decision Tree is:', metrics.accuracy_score(prediction, test_Y))
result=cross_val_score(model, X, Y, cv=10, scoring='accuracy')
print('The cross validated score for bagged Decision Tree is:', result.mean())

#### Boosting
* Boosting은 classifier의 순차적 학습을 사용하는 앙상블 기술이다. 약한 모델을 단계적으로 개선하는 것이다. 부스팅은 다음과 같이 작동한다.
* 모델은 먼저 전체 데이터 세트에서 학습된다. 이제 모델은 일부 인스턴스는 맞고 일부는 틀리게 된다. 다음 반복에서는 학습모델은 잘못 예측된 인스턴스에 더 집중하거나 더 많은 가중치를 부여한다. 따라서 잘못된 인스턴스를 올바르게 예측하려고 시도한다. 이제 이 반복 프로세스가 계속 되고 정확도의 한계에 도달할 때까지 새로운 classifier가 모델에 추가된다.
---
##### AdaBoost(Adaptive Boosting)
* 이 경우 약한 학습모델 또는 추정기는 Decision Tree이다. 그러나 우리는 default base_estimator를 우리가 선택한 알고리즘으로 변경할 수 있다.

In [65]:
from sklearn.ensemble import AdaBoostClassifier
ada=AdaBoostClassifier(n_estimators=200, random_state=0, learning_rate=0.1)
result=cross_val_score(ada, X,Y,cv=10, scoring='accuracy')
print('The cross validated score for AdaBoost is:', result.mean())

##### Stochastic Gradient Boosting
* 여기서도 약한 학습모델은 Decision Tree이다.

In [66]:
from sklearn.ensemble import GradientBoostingClassifier
grad=GradientBoostingClassifier(n_estimators=500, random_state=0, learning_rate=0.1)
result=cross_val_score(grad, X,Y,cv=10,scoring='accuracy')
print('The cross validated score for Gradient Boosting is:', result.mean())

##### XGBoost

In [67]:
import xgboost as xg
xg.set_config(verbosity=0) # 이게 뭘까??
xgboost=xg.XGBClassifier(n_estimators=900, learning_rate=0.1)
result=cross_val_score(xgboost,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for XGBoost is:', result.mean())

* AdaBoost에서 가장 높은 정확도를 얻었다. 하이퍼파라미터 튜닝으로 높여보도록 하자.
---
##### Hyper-Parameter Tuning for AdaBoost

In [68]:
n_estimators=list(range(100,1100,100))
learn_rate=[0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
hyper={'n_estimators':n_estimators, 'learning_rate':learn_rate}
gd=GridSearchCV(estimator=AdaBoostClassifier(), param_grid=hyper, verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

* AdaBoost로 얻을 수 있는 최대 정확도는 83.16%이다. 이때 n_estimator= 이고 learning_rate= 이다.
---
### Confusion Matrix for the Best Model

In [69]:
ada=AdaBoostClassifier(n_estimators=200, random_state=0, learning_rate=0.05)
result=cross_val_predict(ada,X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,result),cmap='winter',annot=True,fmt='2.0f')
plt.show()

### Feature Importance

In [70]:
f,ax=plt.subplots(2,2,figsize=(15,12))
model=RandomForestClassifier(n_estimators=500,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[0,0])
ax[0,0].set_title('Feature Importance in Random Forests')
model=AdaBoostClassifier(n_estimators=200,learning_rate=0.05,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[0,1],color='#ddff11')
ax[0,1].set_title('Feature Importance in AdaBoost')
model=GradientBoostingClassifier(n_estimators=500,learning_rate=0.1,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[1,0],cmap='RdYlGn_r')
ax[1,0].set_title('Feature Importance in Gradient Boosting')
model=xg.XGBClassifier(n_estimators=900,learning_rate=0.1)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[1,1],color='#FD0F00')
ax[1,1].set_title('Feature Importance in XgBoost')
plt.show()

* RandomForest, AdaBoost등과 같은 다양한 분류기의 중요한 특성들을 볼 수 있다.
----
##### Observations:   
1) 일반적인 중요한 특성 중 일부는 Initial, Fare_cat, Pclass, Family_Size이다.   
2) Sex 특성은 중요도를 부여하지 않는 것 같다. 이전에 Pclass와 결합된 Sex가 매우 좋은 차별화 요소를 제공하는 것을 보았듯이 충격적이다. Sex는 RandomForests에서만 중요한 것 같다. 그러나 많은 분류기에서 맨 위에 있는 Initial 특성을 볼 수 있다. 우리는 이미 Sex와 Initial 사이의 양의 상관관계를 보았으므로 둘 다 성별을 나타낸다.   
3) 유사하게 Pclass 및 Fare_cat은 Alone, Parch 및 SibSp와 함께 승객 및 Family_Size의 상태를 나타낸다.